# Automated Layer Analysis Version 3.X
First developed by Nicole S Dawney, Christian Cammarota and Dan Bergstralh 2020-2022 at the University of Rochester
This version and annotations written by Tara M Finegan September 2023 at the University of Missouri

## 1. Initialization
The code in these cells are required to use ALAn. Please run them in order from **1.1** to **1.4** before continuing.

### 1.1: Packages
The code in this cell imports all necessary Python packages and sets Plots to be interactive in the note

In [ ]:
import os                                   
import numpy as np                            
import skimage.io as io            
from scipy import optimize, stats                      
import pandas as pd                             
import math
import inspect
import time
from scipy import signal
from IPython.display import clear_output
import matplotlib.pyplot as plt   
from matplotlib import colors
# %matplotlib notebook
%matplotlib inline

from skimage import filters, measure, morphology, segmentation, util 
from skimage.filters import try_all_threshold
from skimage.feature import peak_local_max, canny        
from scipy import optimize, ndimage              
from skimage.measure import label, regionprops    
from skimage.segmentation import clear_border, watershed
from skimage.color import label2rgb, rgb2gray              
from skimage.morphology import remove_small_objects, remove_small_holes  

 # Module containing all the function code for Alan. 
import ALAN_module as alan
from ALAN_module import *

import ALAN_process
from ALAN_process import batch_process, parse

from custom_module import *

### 1.2: I/O Path
This cell specifies the directories for the input path and retrieves from the data the images that will be analyzed, their corresponding names & DataFrames.

In [ ]:
path = r"C:\Users\Bach Nguyen\Desktop\Alan Inputs\Example data\\"
list_of_names, list_of_dfs, list_of_unshuffled_images = ALAN_process.parse(path)

### 1.3: Batch Processing
This function allows for processing multiple images at once. Detailed descriptions of each image will be output into a pandas DataFrame, and are optionally saved to a .csv file. 

#### Optional Parameters:
- **actin_channel**: (Default: 1) Integer specifying which channel for a multichannel image corresponds to the actin channel.
- **invert**: (Default: false)  Boolean specifying to invert the z slices of an image if the z-stack was taken top to bottom instead of bottom to top.
- **save_name**: A string name of a dictionary of analyzed image properties, in .csv file format. **Defaults to not saving the dictionary.**

*Additionally, this cell displays the DataFrame to help determine the element number for the dataset of interest.*

In [ ]:
analyzed_df = batch_process(list_of_dfs, list_of_unshuffled_images, list_of_names, actin_channel = 1, save_name = "test_ALAN")
display(analyzed_df)

### 1.4: Dataset of Interest
Determine the element number for the dataset of interest from the output .csv file in **1.3**.
Change the 'element_number' integer to continue.

In [ ]:
element_number = 1 # Change this to Analyze your DataFrame of choice
i = element_number

## 2: Diagnostic/Informative plots and visuals.
The following functions are to generate diagnostic data outputs and figures. 
- **Optional:** For any function, an optional parameter of save_name = 'STRING_NAME' will allow you to save these plots in .pdf 

### 2A: Clear Debris
This function provides data on which objects identified by the nuclear segmentation input are considered 'real' cells and used for analysis. A histogram shows the distribution of nuclear volumes in µm³ (x-axis) against the frequency of these volumes. The orange bars represent nuclei identified as 'real' and the blue bars represent 'debris' removed from the analysis. A dotted line indicates the threshold volume above which nuclei is considered real. 
- This function can also be used to output A pandas DataFrame containing the nuclei identified as true datapoints (df_cleared)

#### Optional Parameters for clear_debris
- **plot:** (Default: false) whether or not to display the histogram.
- **cutoff:** (Default: 1.5) If you are not happy with the cutoff volume of 'real' nuclei, you can change the value that is used to define what a real nucleus is in the clear_debris function by adding a 'cutoff' integer parameter that is defaulted at 1.5 standard deviations below the average nuclear volume.

In [ ]:
adf = clear_debris(list_of_dfs[i], plot=True)
display(adf)

### 2B: Nuclear centroid actin overlay
Displays an image that shows the centroid position of the segmented nuclei overlaid onto the .tiff image of the actin signal projected on the XY plane of the tissue.

#### Optional Parameters:
- **actin_channel**: (Default: 1) Integer specifying which channel for a multichannel image corresponds to the actin channel.

In [ ]:
nuclear_centroid_actin_overlay (list_of_dfs[i], list_of_unshuffled_images[i], save_name = False, actin_channel = 1)

### 2C: Nuclei distribution
A plot showing both the projected signal of actin (red) and the nuclear distribution (blue), fit with a gaussian, with respect to the depth of the tissue. The tissue layer top, bottom, and the peak position of the actin signal are shown by dotted lines.

#### Optional Parameters:
- **section**: A tuple with four elements corresponding to the bottom, top, left, and right positions of a region of interest within the greater field of view. **Defaults to False positions** which cause the entire field of view to be analyzed.
- **invert**: (Default: false)  Boolean specifying to invert the z slices of an image if the z-stack was taken top to bottom instead of bottom to top.

In [ ]:
nuclei_distribution(list_of_dfs[i], list_of_unshuffled_images[i], plot = True, actin_channel = 1, invert = False)

### 2D: Find shoulders
Function to detect whether the actin intensity plot has a ‘shoulder’, the exact mechanism to determine the shoulder is described in Dawney and Cammarota et al. 2023. This function returns the number of peaks and the ratio of the peak heights, or Later-to-Apical shape index. It also plots the actin signal (red), and the first derivative of this plot (blue) with respect to the tissue depth. The tissue layer top, bottom, and the peak position of the actin signal are shown by dotted lines.

#### Optional Parameters:
- **actin_channel**: (Default: 1) Integer specifying which channel for a multichannel image corresponds to the actin channel.
- **section**: A tuple with four elements corresponding to the (bottom, top, left, right) positions of a region of interest within the greater field of view. **Defaults** to False positions which cause the entire field of view to be analyzed.
- **invert**: (Default: false)  Boolean specifying to invert the z slices of an image if the z-stack was taken top to bottom instead of bottom to top.

In [ ]:
find_shoulders(list_of_dfs[i], list_of_unshuffled_images[i], plot = True, actin_channel = 1, invert = False)

### 2E: Terrain Map
A map of the position of the centroids of nuclei projected onto the XY plane of the tissue, displayed as dots. The size of the dots show the relative volumes of nuclei. Dot colors represent relative nuclei positions with respect to the tissue depth.
- Color scale goes from Light orange (most basal) to Purple (most apical).
#### Optional Parameters:
- **colorbar**: (Default: False) Specify True to add a colorbar axes to the right of the Plot 

In [ ]:
terrain_map(list_of_dfs[i], list_of_unshuffled_images[i], plot = True, color = 'magma_r', invert = False, save_name = False, colorbar = True)

### 2F: Sub-classify
A map representing the XY field of view of the input 512x512 pixel .tiffs broken into 100x100 pixel sections, color-coded by layer category as determined by the original version of ALAn.

#### Color Key
- lawngreen: Immature
- forestgreen:Intermediate
- steelblue: Mature
- orange: Disorganized

In [ ]:
sub_classify(list_of_dfs[i], list_of_unshuffled_images[i], plot = True, invert = False)

## Testing Custom Modules
DISCLAIMER: I don't know what these do but they were in the module with Runtime errors so I went and fixed them

In [ ]:
fake_layers(list_of_dfs[i], list_of_unshuffled_images[i], plot = True, invert = False, fake_type = 'double')

In [ ]:
z_projection_with_cutoffs(list_of_dfs[i], list_of_unshuffled_images[i], plot = True, invert = False)